# MDS (Multidimensionale Skalierung)

#### Was sind euklidische Abstände?
- Der euklidische Abstand misst die direkte (geradlinige) Distanz zwischen zwei Datenpunkten im n-dimensionalen Raum.
- Ziel: Ähnlichkeit oder Unterschiedlichkeit zwischen Daten zu quantifizieren (z. B. zwischen DSC-Kurven).

#### Was ist MDS (Multidimensionale Skalierung)?
- MDS ist eine Technik zur Visualisierung von Ähnlichkeiten oder Unähnlichkeiten in Daten, basierend auf Distanzmaßen wie dem euklidischen Abstand.
- Ziel: Darstellung hochdimensionaler Daten in einem 2D- oder 3D-Raum, wobei die relative Anordnung der Punkte die Abstände widerspiegelt.

#### Warum euklidische Abstände und MDS für DSC-Messwerte?
- **Quantifizierung der Unterschiede:** Euklidische Abstände geben einen klaren Wert für die Ähnlichkeit zwischen zwei DSC-Kurven.
- **Reduktion und Visualisierung:** MDS reduziert die Dimension der Distanzmatrix und stellt die Beziehungen zwischen den Proben visuell dar.
- **Identifikation von Gruppen:** Ähnliche Kurven werden im MDS-Plot nahe beieinander dargestellt, Unterschiede werden räumlich hervorgehoben.
- **Vergleich komplexer Daten:** DSC-Messwerte mit vielen Datenpunkten können so in einfach verständliche Muster überführt werden.

## 1) Einlesen der Daten

In [1]:
file_path = "../data/030_dsc_experiments.csv"

In [2]:
import pandas as pd
import numpy as np

# Einlesen der CSV-Datei
df_experiments = pd.read_csv(file_path, index_col=0)
df_experiments

## 2) Filtern der relevanten Daten (Segment "_S5")

In [3]:
# Filtern der Spalten, die "_S5" enthalten (Segment 5 "Erhitzen")
columns_s5 = [col for col in df_experiments.columns if "_S5" in col]
df_heating = df_experiments[columns_s5]
df_heating

## 3) Normalisierung der Daten

In [4]:
from sklearn.preprocessing import StandardScaler

# Normalisierung der Daten
scaler = StandardScaler()
data_normalized = scaler.fit_transform(df_heating)

# Umwandlung zurück in ein DataFrame
df_heating_normalized = pd.DataFrame(data_normalized, columns=df_heating.columns, index=df_heating.index)
df_heating_normalized

## 4) Transposition der Datenmatrix

In [5]:
# Transposition der Datenmatrix: Proben werden Zeilen
df_basis = df_heating_normalized.T
df_basis

## 5) Berechnung der Euklidischen Distanzen

In [6]:
from scipy.spatial.distance import pdist, squareform

### Paarweise Ähnlichkeit berechnen

In [7]:
# Beispiel-Datensatz: Zeilen = Proben, Spalten = Temperaturen
data = df_basis.values  # Extrahiere die Werte

# Berechnung der paarweisen Euklidischen Distanzen
distance_matrix = squareform(pdist(data, metric="euclidean"))

# Ausgabe der Distanzmatrix als DataFrame
distance_df = pd.DataFrame(distance_matrix, index=df_basis.index, columns=df_basis.index)

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

# Heatmap der Distanzmatrix erstellen
plt.figure(figsize=(10, 8))
sns.heatmap(distance_df, annot=False, cmap="viridis", cbar=True)
plt.title("Heatmap der paarweisen euklidischen Distanzen")
plt.xlabel("Proben")
plt.ylabel("Proben")
plt.show()

- **Dunkelblau**: Geringe euklidische Distanz (hohe Ähnlichkeit zwischen Proben).  
- **Gelb**: Hohe euklidische Distanz (geringe Ähnlichkeit zwischen Proben).  
- **Diagonale (immer Dunkelblau)**: Distanz einer Probe zu sich selbst (Wert = 0).  
- **Farbverlauf von Dunkelblau nach Gelb**: Visualisiert den Grad der Ähnlichkeit (von hoch zu gering).  

## 6) MDS

In [9]:
from sklearn.manifold import MDS
import matplotlib.pyplot as plt

# MDS-Instanz erstellen (mit explizitem normalized_stress)
mds = MDS(n_components=2, dissimilarity="precomputed", normalized_stress="auto", random_state=42)

# MDS auf die Distanzmatrix anwenden
mds_result = mds.fit_transform(distance_matrix)

# Ergebnisse in einen DataFrame speichern
df_basis["MDS1"] = mds_result[:, 0]
df_basis["MDS2"] = mds_result[:, 1]
df_basis

## 7) Visualisierung der MDS-Ergebnisse

In [10]:
# Visualisierung der MDS-Ergebnisse
plt.figure(figsize=(8, 6))
plt.scatter(df_basis["MDS1"], df_basis["MDS2"], c="blue", edgecolor="k", s=50)
plt.title("MDS: Multidimensionale Skalierung")
plt.xlabel("MDS Dimension 1")
plt.ylabel("MDS Dimension 2")
plt.grid(True)
plt.show()

## 8) Export als CSV

In [11]:
df_basis.index = df_basis.index.astype(str).str.replace('_S5', '', regex=False)
df_basis

In [12]:
df_basis.to_csv("data/mds_simple.csv", index=True)